In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
from snownlp import SnowNLP
from snownlp import sentiment
from snownlp import seg

In [3]:
#噓文需要跟主文章相反，中立文本變成零分的轉換副程式
def change_score(tag,word):
    if '噓' in tag:
        if word==1:
            return -1
        if word==-1:
            return 1
    if '→' in tag:
        return 0
    return word

In [5]:
#讀取六月到十二月熱門文章的回文，並做資料處理
message_hot=pd.read_csv("../data/hot_push.txt")
message_hot['content']=message_hot['content'].apply(lambda x:x.replace(":",""))
message_hot=message_hot[['link','content','tag','author','time']]

#讀取六月到十二月熱門文章貼標
hot_article_score=pd.read_csv('../data/hot_article_tag_by_hand.csv')
hot_article_score=hot_article_score[['link','score']]

#將熱門文章回文進行分數轉換，將轉換分數完的結果，分成看漲文本hot_up、看跌文本hot_down，做為訓練集資料的產生
hot_message_score=pd.merge(message_hot,hot_article_score,how='left',on='link')
hot_message_score['new']=hot_message_score.apply(lambda x: change_score(x['tag'], x['score']), axis=1)
hot_final_score=hot_message_score[['author','time','content','new','tag']]
hot_up=hot_final_score[hot_final_score['new']==1]
hot_up=hot_up['content']
hot_down=hot_final_score[hot_final_score['new']==-1]
hot_down=hot_down['content']

In [7]:
#讀取六月到十二月凱衛的回文
message=pd.read_csv("../data/5201_push.csv")
message=message[['link','content','tag','author','time']]

#讀取六月到十二月凱衛文章貼標
article_score=pd.read_csv('../data/5201_article_by_hand.csv')
article_score=article_score[['link','score']]

#將熱門文章回文進行分數轉換，將轉換分數完的結果，最後data_set的名稱是final_score
message_score=pd.merge(message,article_score,how='left',on='link')
message_score['new']=message_score.apply(lambda x: change_score(x['tag'], x['score']), axis=1)
final_score=message_score[['author','time','content','new','tag']]

#將凱衛看漲文本80%和熱門回文垂直合併，產生最後的訓練集up_train.txt
up=final_score[final_score['new']==1]
up_stop=(len(up)/5)*4
up_train=up[0:up_stop]
up_end=up[up_stop:len(up)]
up_train=up_train['content']
up_train=pd.concat([hot_up,up_train])
up_train.to_csv('../outcome/up_train.txt',index=None,header=None)
#將凱衛看跌文本80%和熱門回文垂直合併，產生最後的訓練集down_train.txt
down=final_score[final_score['new']==-1]
down_stop=(len(down)/5)*4
down_train=down[0:down_stop]
down_end=down[down_stop:len(down)]
down_train=down_train['content']
down_train=pd.concat([down_train,hot_down])
down_train.to_csv('../outcome/down_train.txt',index=None,header=None)

#將剩下看漲與看跌的文本，合併成最後測驗的資料集
verification=pd.concat([up_end,down_end])

In [9]:
sentiment.train(r'../outcome/down_train.txt',
                r'../outcome/up_train.txt')
#這邊要根據作業系統安裝的位置調整
sentiment.save(r'/usr/local/lib/python2.7/site-packages/snownlp/sentiment/sentiment.marshal')

In [10]:
#套用情緒分析模型計算分數
def model(x):
    s=SnowNLP(str(x).decode('utf-8'))
    return s.sentiments
verification['nlp_score']=verification['content'].apply(model)

In [11]:
#只要大於0.5就算是看漲，小於0.5就是看跌，等於0.5中立代表不預測
def convert(x):
    if x>0.51:
        return 1
    if x==0.5:
        return 0
    if x<=0.49:
        return -1
verification['convert']=verification['nlp_score'].apply(convert)

In [12]:
#計算命中率
def hit(x,y):
    if x==y:
        return 1
    if x != y:
        return 0

verification['hit'] = verification.apply(lambda x: hit(x['new'], x['convert']), axis=1)

#計算所有上漲的文本數量
all_up=verification.loc[verification['new']==1]
#計算預測上漲的文本數量
model_up=verification.loc[verification['convert']==1]
#有猜中上漲的文本數量
hit_up=verification[(verification['hit']==1) & (verification['new']==1)]
#預測上漲涵蓋率
print "涵蓋率"+str(float(len(hit_up))/len(all_up))
#預測上漲精準度
print "精準度"+str(float(len(hit_up))/len(model_up))


#計算所有下跌的文本數量
all_down=verification.loc[verification['new']==-1]
#計算預測下跌的文本數量
model_down=verification.loc[verification['convert']==-1]
#有猜中下跌的文本數量
hit_down=verification[(verification['hit']==1) & (verification['new']==-1)]
#預測下跌涵蓋率
print "涵蓋率"+str(float(len(hit_down))/len(all_down))
#預測下跌精準度
print "精準度"+str(float(len(hit_down))/len(model_down))

print '----------------------------------------------------------------'
#整體涵蓋率
print '整體涵蓋率'+str(float((len(hit_down))+len(hit_up))/(len(all_down)+(len(all_up))))
#整體精確度
print '整體精確率'+str(float((len(hit_down))+len(hit_up))/(len(model_down)+(len(model_up))))

涵蓋率0.244131455399
精準度0.536082474227
涵蓋率0.813793103448
精準度0.60824742268
----------------------------------------------------------------
整體涵蓋率0.572564612326
整體精確率0.59381443299
